## Урок 10. Машинный перевод. Модель seq2seq и механизм внимания

Разобраться с моделькой перевода как она устроена
запустить для перевода с русского на английский (при желании можно взять другие пары языков) два варианта 

 - с вниманием
 - без внимания

оценить качество насколько корректно переводит (для теста отобрать примеры с увеличением длины текста) (так как оценка визуальная достаточно 20-ти примеров в тестовой выборке)

In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import pandas as pd
import unicodedata
import re
import numpy as np
import os
import io
import time

In [2]:
!wget http://www.manythings.org/anki/rus-eng.zip

--2022-06-24 17:45:39--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.186.54, 104.21.92.44, 2606:4700:3030::6815:5c2c, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.186.54|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14819554 (14M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  14.13M  8.82MB/s    in 1.6s    

2022-06-24 17:45:41 (8.82 MB/s) - ‘rus-eng.zip’ saved [14819554/14819554]



In [3]:
!mkdir rus-eng
!unzip rus-eng.zip -d rus-eng/

Archive:  rus-eng.zip
  inflating: rus-eng/rus.txt         
  inflating: rus-eng/_about.txt      


In [4]:
!ls /content/rus-eng/ -lah

total 71M
drwxr-xr-x 2 root root 4.0K Jun 24 17:45 .
drwxr-xr-x 1 root root 4.0K Jun 24 17:45 ..
-rw-r--r-- 1 root root 1.5K May  2 01:29 _about.txt
-rw-r--r-- 1 root root  71M May  2 01:29 rus.txt


In [5]:
# Download the file
path_to_file = "/content/rus-eng/rus.txt"

In [6]:
def preprocess_sentence(w):
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [7]:
preprocess_sentence("I can't go.")

"<start> i can't go . <end>"

In [8]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENG, RUS]
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [9]:
en, ru = create_dataset(path_to_file, None)
print(en[77])
print(ru[77])

<start> smile . <end>
<start> улыбнись . <end>


In [10]:
print(en[10])
print(ru[10])

<start> run ! <end>
<start> бегите ! <end>


In [11]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [12]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

### Limit the size of the dataset to experiment faster (optional)

In [13]:
len(en), len(ru)

(444587, 444587)

In [14]:
# Try experimenting with the size of that dataset
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [15]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

80000 80000 20000 20000


In [16]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [17]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
394 ----> подожди
544 ----> снаружи
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
114 ----> wait
310 ----> outside
3 ----> .
2 ----> <end>


### Create a tf.data dataset

In [18]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [19]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 15]), TensorShape([64, 11]))

In [20]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=False,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [21]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_hidden = encoder(example_input_batch, sample_hidden)
# print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder Hidden state shape: (batch size, units) (64, 1024)


## Decoder without attention (без внимания).

In [22]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

  def call(self, x, hidden):
    # enc_output shape == (batch_size, max_length, hidden_size)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x, initial_state=hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state

In [23]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden)

In [24]:
decoder_sample_x.shape

TensorShape([64, 7334])

In [25]:
decoder_sample_h.shape

TensorShape([64, 1024])

## Define the optimizer and the loss function

In [26]:
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [27]:
checkpoint_dir = './training_nmt_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [28]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden = decoder(dec_input, dec_hidden)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [29]:
EPOCHS = 50

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.6271
Epoch 1 Batch 100 Loss 1.9635
Epoch 1 Batch 200 Loss 1.7842
Epoch 1 Batch 300 Loss 1.6588
Epoch 1 Batch 400 Loss 1.4527
Epoch 1 Batch 500 Loss 1.4227
Epoch 1 Batch 600 Loss 1.3686
Epoch 1 Batch 700 Loss 1.1667
Epoch 1 Batch 800 Loss 1.1411
Epoch 1 Batch 900 Loss 1.1840
Epoch 1 Batch 1000 Loss 1.0534
Epoch 1 Batch 1100 Loss 0.9597
Epoch 1 Batch 1200 Loss 1.0568
Epoch 1 Loss 1.4100
Time taken for 1 epoch 66.63047742843628 sec

Epoch 2 Batch 0 Loss 0.8692
Epoch 2 Batch 100 Loss 0.8615
Epoch 2 Batch 200 Loss 0.8151
Epoch 2 Batch 300 Loss 0.8157
Epoch 2 Batch 400 Loss 0.8053
Epoch 2 Batch 500 Loss 0.7825
Epoch 2 Batch 600 Loss 0.7990
Epoch 2 Batch 700 Loss 0.6812
Epoch 2 Batch 800 Loss 0.7372
Epoch 2 Batch 900 Loss 0.6095
Epoch 2 Batch 1000 Loss 0.5320
Epoch 2 Batch 1100 Loss 0.4811
Epoch 2 Batch 1200 Loss 0.5499
Epoch 2 Loss 0.6863
Time taken for 1 epoch 56.90160369873047 sec

Epoch 3 Batch 0 Loss 0.3805
Epoch 3 Batch 100 Loss 0.4325
Epoch 3 Batch 200 Loss 0.417

## Translate

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [30]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden = decoder(dec_input, dec_hidden)

    # storing the attention weights to plot later on
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [31]:
def remove_char(s):
    remove_result = s[8 : -6]
    return remove_result

def remove_char_trans(s):
    remove_result_trans = s[0 : -6]
    return remove_result_trans

In [32]:
final_result = []

def translate(sentence):
  result, sentence = evaluate(sentence)

  sent = (remove_char(sentence))
  res = (remove_char_trans(result))
  
  final_result.append([sent, res])
  df = pd.DataFrame(final_result, columns=['текст', 'перевод без внимания'])

  return df

## Restore the latest checkpoint and test

In [33]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [34]:
translate('Добрый вечер!')

,текст,перевод без внимания
0,добрый вечер !,good evening .


In [35]:
translate('не верь глазам своим.')

,текст,перевод без внимания
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't tell our hands .


In [36]:
translate('Я люблю, когда идет снег.')

,текст,перевод без внимания
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't tell our hands .
2,"я люблю , когда идет снег .",i like snow tonight .


In [37]:
translate('Пора ложиться спать.')

,текст,перевод без внимания
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't tell our hands .
2,"я люблю , когда идет снег .",i like snow tonight .
3,пора ложиться спать .,it's time for to bed .


In [38]:
translate('Пора ложиться спать, хотя ещё не поздно.')

,текст,перевод без внимания
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't tell our hands .
2,"я люблю , когда идет снег .",i like snow tonight .
3,пора ложиться спать .,it's time for to bed .
4,"пора ложиться спать , хотя ещ не поздно .",it's late or not out .


In [39]:
translate('Этот переводчик, знает очень мало слов.')

,текст,перевод без внимания
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't tell our hands .
2,"я люблю , когда идет снег .",i like snow tonight .
3,пора ложиться спать .,it's time for to bed .
4,"пора ложиться спать , хотя ещ не поздно .",it's late or not out .
5,"этот переводчик , знает очень мало слов .","this soup , too it ."


In [40]:
translate('Этот переводчик, знает очень мало слов и работает не точно.')

,текст,перевод без внимания
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't tell our hands .
2,"я люблю , когда идет снег .",i like snow tonight .
3,пора ложиться спать .,it's time for to bed .
4,"пора ложиться спать , хотя ещ не поздно .",it's late or not out .
5,"этот переводчик , знает очень мало слов .","this soup , too it ."
6,"этот переводчик , знает очень мало слов и рабо...",it suits you food .


In [41]:
translate('Чем больше слов, тем хуже перевод.')

,текст,перевод без внимания
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't tell our hands .
2,"я люблю , когда идет снег .",i like snow tonight .
3,пора ложиться спать .,it's time for to bed .
4,"пора ложиться спать , хотя ещ не поздно .",it's late or not out .
5,"этот переводчик , знает очень мало слов .","this soup , too it ."
6,"этот переводчик , знает очень мало слов и рабо...",it suits you food .
7,"чем больше слов , тем хуже перевод .",any child seems wrong .


In [42]:
translate('Спасибо за внимание!')

,текст,перевод без внимания
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't tell our hands .
2,"я люблю , когда идет снег .",i like snow tonight .
3,пора ложиться спать .,it's time for to bed .
4,"пора ложиться спать , хотя ещ не поздно .",it's late or not out .
5,"этот переводчик , знает очень мало слов .","this soup , too it ."
6,"этот переводчик , знает очень мало слов и рабо...",it suits you food .
7,"чем больше слов , тем хуже перевод .",any child seems wrong .
8,спасибо за внимание !,thanks for the tip .


In [43]:
df1 = pd.DataFrame(final_result)

In [44]:
df1.to_csv('df1.csv') # сохраним на всякий случай

## Decoder with attention (с вниманием).

In [45]:
def preprocess_sentence(w):
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [46]:
preprocess_sentence("I can't go.")

"<start> i can't go . <end>"

In [47]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENG, RUS]
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [48]:
en, ru = create_dataset(path_to_file, None)
print(en[-1])
print(ru[-1])

<start> doubtless there exists in this world precisely the right woman for any given man to marry and vice versa but when you consider that a human being has the opportunity of being acquainted with only a few hundred people , and out of the few hundred that there are but a dozen or less whom he knows intimately , and out of the dozen , one or two friends at most , it will easily be seen , when we remember the number of millions who inhabit this world , that probably , since the earth was created , the right man has never yet met the right woman . <end>
<start> несомненно , для каждого мужчины в этом мире где то есть подходящая женщина , которая может стать ему женой , обратное верно и для женщин . но если учесть , что у человека может быть максимум несколько сотен знакомых , из которых лишь дюжина , а то и меньше , тех , кого он знает близко , а из этой дюжины у него один или от силы два друга , то можно легко увидеть , что с уч том миллионов живущих на земле людей , ни один подходящи

In [49]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [50]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [51]:
len(en), len(ru)

(444587, 444587)

In [52]:
# Try experimenting with the size of that dataset
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [53]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

80000 80000 20000 20000


In [54]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [55]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
2102 ----> встаньте
9 ----> ,
6 ----> том
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
422 ----> stand
54 ----> up
39 ----> ,
5 ----> tom
3 ----> .
2 ----> <end>


In [56]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [57]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 15]), TensorShape([64, 11]))

In [58]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [59]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 15, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [60]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [61]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 15, 1)


In [62]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [63]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 7334)


In [64]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [65]:
checkpoint_dir = './training_attention_checkpoints2'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [66]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [67]:
EPOCHS = 50

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.6654
Epoch 1 Batch 100 Loss 2.1782
Epoch 1 Batch 200 Loss 1.8577
Epoch 1 Batch 300 Loss 1.7241
Epoch 1 Batch 400 Loss 1.6300
Epoch 1 Batch 500 Loss 1.5560
Epoch 1 Batch 600 Loss 1.3881
Epoch 1 Batch 700 Loss 1.3256
Epoch 1 Batch 800 Loss 1.2855
Epoch 1 Batch 900 Loss 1.2683
Epoch 1 Batch 1000 Loss 1.0287
Epoch 1 Batch 1100 Loss 1.0638
Epoch 1 Batch 1200 Loss 1.0413
Epoch 1 Loss 1.4618
Time taken for 1 epoch 111.69122433662415 sec

Epoch 2 Batch 0 Loss 0.8359
Epoch 2 Batch 100 Loss 0.8989
Epoch 2 Batch 200 Loss 0.7823
Epoch 2 Batch 300 Loss 0.8763
Epoch 2 Batch 400 Loss 0.6828
Epoch 2 Batch 500 Loss 0.6905
Epoch 2 Batch 600 Loss 0.5663
Epoch 2 Batch 700 Loss 0.6700
Epoch 2 Batch 800 Loss 0.5771
Epoch 2 Batch 900 Loss 0.5721
Epoch 2 Batch 1000 Loss 0.5262
Epoch 2 Batch 1100 Loss 0.4760
Epoch 2 Batch 1200 Loss 0.4392
Epoch 2 Loss 0.6345
Time taken for 1 epoch 101.04877924919128 sec

Epoch 3 Batch 0 Loss 0.3799
Epoch 3 Batch 100 Loss 0.3073
Epoch 3 Batch 200 Loss 0.3

In [68]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result2 = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result2 += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result2, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result2, sentence, attention_plot

In [69]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [143]:
final_result2 = []

def remove_char(s):
    remove_result = s[8 : -6]
    return remove_result

def remove_char_trans(s):
    remove_result_trans = s[0 : -6]
    return remove_result_trans

def translate(sentence):
  result2, sentence, attention_plot = evaluate(sentence)

  sent = (remove_char(sentence))
  res = (remove_char_trans(result2))
  
  final_result2.append([sent, res])
  df2 = pd.DataFrame(final_result2, columns=['текст', 'перевод с вниманием'])

  # print('Input: %s' % (sentence))
  # print('Predicted translation: {}'.format(result2))

  attention_plot = attention_plot[:len(result2.split(' ')), :len(sentence.split(' '))]
  # plot_attention(attention_plot, sentence.split(' '), result.split(' '))
  return df2

In [138]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [144]:
translate('Добрый вечер!')

,текст,перевод с вниманием
0,добрый вечер !,good evening .


In [145]:
translate('не верь глазам своим.')

,текст,перевод с вниманием
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't buy your suit .


In [146]:
translate('Я люблю, когда идет снег.')

,текст,перевод с вниманием
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't buy your suit .
2,"я люблю , когда идет снег .",i like to drink snow .


In [147]:
translate('Пора ложиться спать.')

,текст,перевод с вниманием
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't buy your suit .
2,"я люблю , когда идет снег .",i like to drink snow .
3,пора ложиться спать .,it's time for bed .


In [148]:
translate('Пора ложиться спать, хотя ещё не поздно.')

,текст,перевод с вниманием
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't buy your suit .
2,"я люблю , когда идет снег .",i like to drink snow .
3,пора ложиться спать .,it's time for bed .
4,"пора ложиться спать , хотя ещ не поздно .",it's time for bed .


In [149]:
translate('Этот переводчик, знает очень мало слов.')

,текст,перевод с вниманием
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't buy your suit .
2,"я люблю , когда идет снег .",i like to drink snow .
3,пора ложиться спать .,it's time for bed .
4,"пора ложиться спать , хотя ещ не поздно .",it's time for bed .
5,"этот переводчик , знает очень мало слов .",this should know tastes .


In [150]:
translate('Этот переводчик, знает очень мало слов и работает не точно.')

,текст,перевод с вниманием
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't buy your suit .
2,"я люблю , когда идет снег .",i like to drink snow .
3,пора ложиться спать .,it's time for bed .
4,"пора ложиться спать , хотя ещ не поздно .",it's time for bed .
5,"этот переводчик , знает очень мало слов .",this should know tastes .
6,"этот переводчик , знает очень мало слов и рабо...",this should know is this work .


In [151]:
translate('Чем больше слов, тем хуже перевод.')

,текст,перевод с вниманием
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't buy your suit .
2,"я люблю , когда идет снег .",i like to drink snow .
3,пора ложиться спать .,it's time for bed .
4,"пора ложиться спать , хотя ещ не поздно .",it's time for bed .
5,"этот переводчик , знает очень мало слов .",this should know tastes .
6,"этот переводчик , знает очень мало слов и рабо...",this should know is this work .
7,"чем больше слов , тем хуже перевод .","god is up , but ."


In [152]:
translate('Спасибо за внимание!')

,текст,перевод с вниманием
0,добрый вечер !,good evening .
1,не верь глазам своим .,don't buy your suit .
2,"я люблю , когда идет снег .",i like to drink snow .
3,пора ложиться спать .,it's time for bed .
4,"пора ложиться спать , хотя ещ не поздно .",it's time for bed .
5,"этот переводчик , знает очень мало слов .",this should know tastes .
6,"этот переводчик , знает очень мало слов и рабо...",this should know is this work .
7,"чем больше слов , тем хуже перевод .","god is up , but ."
8,спасибо за внимание !,thanks for sale .


In [165]:
df2 = pd.read_csv('df2.csv', index_col=0)

In [166]:
df2 = df2.rename({'0': 'текст', '1': 'перевод c вниманием'}, axis=1)

In [167]:
df1 = pd.read_csv('df1.csv', index_col=0)

In [168]:
df1 = df1.rename({'0': 'текст', '1': 'перевод без внимания'}, axis=1)

## Подведём итог

In [170]:
df3 = df1.merge(df2)
df3

,текст,перевод без внимания,перевод c вниманием
0,добрый вечер !,good evening .,good evening .
1,не верь глазам своим .,don't tell our hands .,don't buy your suit .
2,"я люблю , когда идет снег .",i like snow tonight .,i like to drink snow .
3,пора ложиться спать .,it's time for to bed .,it's time for bed .
4,"пора ложиться спать , хотя ещ не поздно .",it's late or not out .,it's time for bed .
5,"этот переводчик , знает очень мало слов .","this soup , too it .",this should know tastes .
6,"этот переводчик , знает очень мало слов и рабо...",it suits you food .,this should know is this work .
7,"чем больше слов , тем хуже перевод .",any child seems wrong .,"god is up , but ."
8,спасибо за внимание !,thanks for the tip .,thanks for sale .


### Большой разницы в качестве перевода "с вниманием" и "без внимания" нет. Качество низкое.